## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path

In [6]:
def submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.ci.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results'
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown'
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [3]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(200)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=26096b84-8376-45f5-9892-e90689bc085c
Running
Running
ara-unsecret Running 0
Running
kp-cam Running 0
Running
kp-textmining Running 0
Running
ara-aragorn Running 0
Running
ara-arax Running 0
Running
kp-chp Running 0
Running
kp-molecular Running 0
Done
ara-explanatory Done 0
Running
kp-icees Running 0
Done
ara-improving ARS Error 0
Running
ara-bte Running 0
Running
kp-openpredict Running 0
Done
kp-cohd ARS Error 0
Error
kp-genetics Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=cd380ba8-7688-4b1c-9156-8fc73c1f2e7a
Running
Running
ara-arax Running 0
Running
kp-cam Running 0
Done
ara-improving ARS Error 0
Running
kp-icees Running 0
Error
ara-aragorn Error 0
Error
kp-chp Error 0
Error
kp-cohd Error 0
Running
kp-textmining Running 0
Running
kp-openpredict R

https://arax.ncats.io/?source=ARS&id=49bfbedf-7864-4eaf-8225-4bdc3a1943c1
Running
Running
ara-aragorn Running 0
Running
kp-icees Running 0
Error
kp-genetics Error 0
Running
kp-textmining Running 0
Running
ara-bte Running 0
Done
ara-improving ARS Error 0
Running
kp-openpredict Running 0
Running
ara-unsecret Running 0
Running
kp-chp Running 0
Done
ara-explanatory Done 0
Running
ara-arax Running 0
Error
kp-cohd Error 0
Error
kp-molecular Error 0
Running
kp-cam Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.4_tryptophan-kynurenine-with-curie.json
D.4_tryptophan-kynurenine-with-curie
https://arax.ncats.io/?source=ARS&id=395c7cd3-60b7-4975-9a12-1be0e64ae931
Running
Running
kp-openpredict Running 0
Running
ara-aragorn Running 0
Error
ara-bte Error 0
Running
ara-arax Running 0
Error
kp-molecular Error 0
Error
kp-cohd Error 0
Running
kp-chp Running 0
Running
kp-textmining Running 0
Running
kp-cam Running 0
Done
kp-icees ARS Error 0
Done
ara-improving ARS Error

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [7]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 26096b84-8376-45f5-9892-e90689bc085c
Done
Error
ara-unsecret Error 0
Done
kp-cam Done 0
Done
kp-textmining Done 0
Error
ara-aragorn Error 0
Done
ara-arax Done 68
Unknown
kp-chp Unknown 0
Done
kp-molecular Done 99
Done
ara-explanatory Done 0
Done
kp-icees Done 0
Done
ara-improving ARS Error 0
Done
ara-bte Done 40
Done
kp-openpredict Done 0
Done
kp-cohd ARS Error 0
Error
kp-genetics Error 0
A.8_EGFR_simple cd380ba8-7688-4b1c-9156-8fc73c1f2e7a
Done
Done
ara-arax Done 336
Done
kp-cam Done 0
Done
ara-improving ARS Error 0
Done
kp-icees Done 0
Error
ara-aragorn Error 0
Error
kp-chp Error 0
Error
kp-cohd Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Error
ara-unsecret Error 0
Done
ara-bte Done 0
Done
ara-explanatory Done 0
Error
kp-genetics Error 0
Done
kp-molecular Done 0
A.2_RHOBTB2_twohop 2a6a79e3-5662-4961-9375-79c599a2d954
Done
Done
kp-textmining Done 0
Error
kp-chp Error 0
Done
kp-openpredict Done 0
Error
kp-cohd Error 0
Done
ara-aragorn Done 0
Done
ara-explana

Done
Done
ara-improving ARS Error 0
Unknown
kp-chp Unknown 0
Error
ara-bte Error 0
Error
kp-cohd Error 0
Error
kp-genetics Error 0
Error
kp-cam Error 0
Error
ara-aragorn Error 0
Error
kp-textmining Error 0
Error
ara-unsecret Error 0
Error
ara-arax Error 0
Done
ara-explanatory Done 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Error
kp-molecular Error 0
C.2_ChemSubstances_related_to_GeneSet 0e17449f-0b84-44df-b015-af9f6cf677fd
Done
Done
ara-explanatory Done 0
Error
kp-molecular Error 0
Done
kp-icees ARS Error 0
Error
kp-cam Error 0
Error
kp-genetics Error 0
Error
ara-bte Error 0
Error
ara-aragorn Error 0
Done
kp-openpredict Done 0
Done
ara-improving ARS Error 0
Error
kp-cohd Error 0
Unknown
kp-chp Unknown 0
Error
ara-unsecret Error 0
Error
kp-textmining Error 0
Error
ara-arax Error 0
C.1_ChemSubstances_related_to_Disease 7a865db0-7ffd-4502-8921-e331bc9141cc
Done
Done
kp-openpredict Done 0
Done
ara-explanatory Done 0
Done
ara-aragorn Done 0
Done
ara-improving ARS Error 0
Error
a

In [8]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine-with-curie
D.5_tryptophan-kynurenine
D.6_metformin-ferritin


In [9]:
df

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=9b079596-...,https://arax.ncats.io/?source=ARS&id=2a6a79e3-...,https://arax.ncats.io/?source=ARS&id=26096b84-...,https://arax.ncats.io/?source=ARS&id=cd380ba8-...,https://arax.ncats.io/?source=ARS&id=172c88cd-...,https://arax.ncats.io/?source=ARS&id=9423d0e4-...,https://arax.ncats.io/?source=ARS&id=ee3ec153-...,https://arax.ncats.io/?source=ARS&id=746c54a1-...,https://arax.ncats.io/?source=ARS&id=49bfbedf-...,https://arax.ncats.io/?source=ARS&id=b148e978-...,...,https://arax.ncats.io/?source=ARS&id=e31dd3f4-...,https://arax.ncats.io/?source=ARS&id=07de4753-...,https://arax.ncats.io/?source=ARS&id=7a865db0-...,https://arax.ncats.io/?source=ARS&id=0e17449f-...,https://arax.ncats.io/?source=ARS&id=518bab34-...,https://arax.ncats.io/?source=ARS&id=0cc20ce7-...,https://arax.ncats.io/?source=ARS&id=c7cea2ce-...,https://arax.ncats.io/?source=ARS&id=395c7cd3-...,https://arax.ncats.io/?source=ARS&id=c4cbc55c-...,https://arax.ncats.io/?source=ARS&id=ba14e04a-...
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-icees,No Results,ARS Error,No Results,No Results,No Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,No Results,No Results,No Results,ARS Error,ARS Error,ARS Error,No Results,ARS Error,ARS Error,ARS Error
kp-genetics,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,No Results,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-openpredict,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
kp-chp,Unknown,Error,Unknown,Error,Unknown,Error,Error,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
ara-improving,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,ARS Error,ARS Error,ARS Error,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-textmining,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,Error,Error,No Results,Error,Error,Error,Error,Error


<br>

## Converting the Pk's to hyperlink

<br>

In [10]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [11]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [12]:
df.sort_index(inplace=True)

In [13]:
df

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Results,No Results,Error,Error,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,No Results,Error,No Results,Results,Error,Error
ara-arax,Results,Results,Results,Results,Error,Results,Error,Error,No Results,Error,...,Results,Results,Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,ARS Error,ARS Error,ARS Error,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,Unknown,Error,Unknown,Error,Unknown,Error,Error,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,No Results,Error,Error,Error,Error,Error,Error,Error


<br>

### Highlight the cells In Excel

<br>

In [14]:
def highlight(v):
    if v=='Results':
        return 'background-color: %s' % 'green'
    elif v=='Error':
        return 'background-color: %s' % 'red'
    elif v == 'No Results':
        return 'background-color: %s' % 'yellow'
    elif v == 'ARS Error':
        return 'background-color: %s' % 'blue'
    else:
        return

In [15]:
styled = df.style.applymap(highlight)

In [16]:
styled

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Results,No Results,Error,Error,Error,No Results,No Results,No Results,No Results,No Results,No Results,Error,Results,Results,No Results,Error,No Results,Error,No Results,Results,Error,Error
ara-arax,Results,Results,Results,Results,Error,Results,Error,Error,No Results,Error,Results,Error,Results,Results,Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,Error,ARS Error,ARS Error,ARS Error,ARS Error,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,Unknown,Error,Unknown,Error,Unknown,Error,Error,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,No Results,Error,Error,Error,Error,Error,Error,Error


In [17]:
styled.to_excel('ara_worklow_progress_tracker_Aug11.xlsx')

In [ ]:
### Save as excel sheet
#df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [ ]:
### save all the ids for ara submission
#with open('ara_worklow_IDs_Aug9.txt', 'w') as f:
#    print(dict_workflows, file=f)

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [18]:
styled

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Results,No Results,Error,Error,Error,No Results,No Results,No Results,No Results,No Results,No Results,Error,Results,Results,No Results,Error,No Results,Error,No Results,Results,Error,Error
ara-arax,Results,Results,Results,Results,Error,Results,Error,Error,No Results,Error,Results,Error,Results,Results,Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Error,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,Error,ARS Error,ARS Error,ARS Error,ARS Error,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,Unknown,Error,Unknown,Error,Unknown,Error,Error,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,No Results,Error,Error,Error,Error,Error,Error,Error


In [19]:
df

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine-with-curie,D.5_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Results,No Results,Error,Error,Error,No Results,No Results,No Results,No Results,No Results,...,Results,Results,No Results,Error,No Results,Error,No Results,Results,Error,Error
ara-arax,Results,Results,Results,Results,Error,Results,Error,Error,No Results,Error,...,Results,Results,Results,Error,Results,Error,Results,Error,Error,Error
ara-bte,Results,No Results,Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,Results,No Results,Error,Error,Error,Error,Error,Error,Error,Error
ara-explanatory,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results
ara-improving,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error,...,ARS Error,ARS Error,ARS Error,ARS Error,Results,ARS Error,ARS Error,ARS Error,ARS Error,ARS Error
ara-unsecret,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-cam,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,No Results,...,No Results,Results,Error,Error,Error,Error,Error,Error,Error,Error
kp-chp,Unknown,Error,Unknown,Error,Unknown,Error,Error,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
kp-cohd,Error,Error,ARS Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error
kp-genetics,Error,Error,Error,Error,Error,Error,Error,Error,Error,Error,...,Error,Error,No Results,Error,Error,Error,Error,Error,Error,Error


<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

In [24]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [25]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [26]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
wks_name = 'Workflow Progress Tracker August 11'
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker August 11' id:1551619533>

In [ ]:
spreadsheet_key

In [27]:
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [ ]:
pwd

In [29]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
sh = gc.open("workflow_progress_tracker").sheet1

In [30]:
sh

<Worksheet 'Workflow Progress Tracker August 11' id:1551619533>

In [31]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W16', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [34]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1551619533,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1551619533,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold'

In [32]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1551619533,
       'startRowIndex': 1,
       'endRowIndex': 16,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [33]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1551619533,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1551619533,
       'startRowIndex': 1,
       'endRowIndex': 16,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorSt

In [35]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:W15', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1551619533,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1551619533,
       'startRowIndex': 1,
       'endRowIndex': 15,
       'startColumnIndex': 1,
       'endColumnIndex': 23}],
     'booleanRule': {'condition': {'type': 'TEXT_EQ',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbCo

In [36]:
# Select a range
cell_list = sh.range('B16:W16')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker August 11'!B16:W16",
 'updatedRows': 1,
 'updatedColumns': 22,
 'updatedCells': 22}